final outputs:

#### ERP + household + school_count + PTV -> by sa2
SA2_info
- ../data/curated/sa2_info.csv

#### ERP + price_index + interest_rate + price_index -> year, SA2
history_data
- ../data/curated/history_data.csv

#### Rent data:
- ../data/curated/rent.csv

#### PTV data
- ../data/raw/PTV/public_trans.csv


In [1]:
import os
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
import re
from itertools import compress
import pandas as pd
import geopandas as gpd
import zipfile


spark = (
    SparkSession.builder.appName("Assignment_2")
    .config("spark.sql.repl.eagerEval.enabled", True)
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .getOrCreate()
)

INPUT_DIR = "../data/raw/"
OUTPUT_DIR = "../data/curated/"


headers = {"accept": "text/csv"}

In [2]:
def gpd_station_merge(poly_gdf, file_path, by_id_name = "SA2_CODE21",\
    station_id_name = "STOP_ID", method={"STOP_ID": "count"}):
    """
        A function used to merge shape file in path: {file_path} to a 
        geopandas dataframe {poly_gdf} with POLYGON geometry. 
        poly_gdf: a geopandas.GeoDataFrame object contains POLYGON geometry
        file_path: a String of file path to read target shape file
        by_id_name: a String of id name to perform groupby option
        station_id_name: a String of id name stated in the readed gdf
        method: a Dict of operations to perform after groupby
    """

    ### read station file
    station_gdf = gpd.read_file(file_path)

    # metro bus station feature selection
    station_gdf = station_gdf[[station_id_name, "geometry"]]
    


    # merge tabels
    join_gdf = gpd.sjoin(poly_gdf, station_gdf, how="left")
    join_gdf = join_gdf.groupby(by_id_name).agg(method)
    
    return join_gdf

## PTV data

In [3]:
output_dir = "../data/raw/PTV/ll_gda2020/esrishape/whole_of_dataset/victoria/PUBLIC_TRANSPORT/"

# unzip zip file
with zipfile.ZipFile(f"../data/raw/PTV.zip", "r") as zip_ref:
    zip_ref.extractall(f"../data/raw/PTV/")

In [4]:
### read shape file and make geometry readable
boundary_gdf = gpd.read_file("../data/raw/ABS/digitalBoundary/\
SA2_2021_AUST_GDA2020.shp")
boundary_gdf = boundary_gdf.loc[boundary_gdf["STE_NAME21"] == "Victoria"]

# digital boundary feature selection
boundary_gdf = boundary_gdf.reset_index()[["SA2_CODE21", "geometry"]].set_index("SA2_CODE21")
print(boundary_gdf.shape)
print(boundary_gdf.head(1))


(524, 1)
                                                     geometry
SA2_CODE21                                                   
201011001   POLYGON ((143.78282 -37.56666, 143.75558 -37.5...


In [5]:
mix_gdf = boundary_gdf.sort_values(by=["SA2_CODE21"])
file_paths = [
    f"{output_dir}PTV_METRO_BUS_STOP.shp",
    f"{output_dir}PTV_METRO_TRAIN_STATION.shp",
    f"{output_dir}PTV_METRO_TRAM_STOP.shp",
    f"{output_dir}PTV_REGIONAL_BUS_STOP.shp",
    f"{output_dir}PTV_REGIONAL_COACH_STOP.shp",
    f"{output_dir}PTV_REGIONAL_TRAIN_STATION.shp",
    f"{output_dir}PTV_SKYBUS_STOP.shp"
]

col_names = ["metrobus_count", "metrotrain_count", "metrotram_count",
    "regbus_count", "regcoach_count", "regtrain_count", "skybus_count"]


for col_name, file_path in zip(col_names, file_paths):
    print(col_name, file_path)
    cur_gdf = gpd_station_merge(boundary_gdf, file_path).rename({"STOP_ID": col_name}, axis=1)
    cur_gdf = cur_gdf.sort_values(by = col_name)
    mix_gdf = pd.concat([mix_gdf, cur_gdf], axis=1)

print(mix_gdf.shape)
print(mix_gdf.head())
mix_gdf.to_csv("../data/raw/PTV/public_trans.csv")

metrobus_count ../data/raw/PTV/ll_gda2020/esrishape/whole_of_dataset/victoria/PUBLIC_TRANSPORT/PTV_METRO_BUS_STOP.shp
metrotrain_count ../data/raw/PTV/ll_gda2020/esrishape/whole_of_dataset/victoria/PUBLIC_TRANSPORT/PTV_METRO_TRAIN_STATION.shp
metrotram_count ../data/raw/PTV/ll_gda2020/esrishape/whole_of_dataset/victoria/PUBLIC_TRANSPORT/PTV_METRO_TRAM_STOP.shp
regbus_count ../data/raw/PTV/ll_gda2020/esrishape/whole_of_dataset/victoria/PUBLIC_TRANSPORT/PTV_REGIONAL_BUS_STOP.shp
regcoach_count ../data/raw/PTV/ll_gda2020/esrishape/whole_of_dataset/victoria/PUBLIC_TRANSPORT/PTV_REGIONAL_COACH_STOP.shp
regtrain_count ../data/raw/PTV/ll_gda2020/esrishape/whole_of_dataset/victoria/PUBLIC_TRANSPORT/PTV_REGIONAL_TRAIN_STATION.shp
skybus_count ../data/raw/PTV/ll_gda2020/esrishape/whole_of_dataset/victoria/PUBLIC_TRANSPORT/PTV_SKYBUS_STOP.shp
(524, 8)
                                                     geometry  metrobus_count  \
SA2_CODE21                                                        

## SA2_info
#### ERP + household + school_count + PTV -> by sa2
Read and prepare all data sets and create TempView

In [6]:
# read school file
# school_sdf = spark.read.csv(f"{INPUT_DIR}ACARA/School_location/School_location.csv", header=True)
school_sdf = spark.read.csv(f"{INPUT_DIR}ACARA/School_location.csv", header=True)
# count school by SA2
school_count = school_sdf.groupBy("SA2").agg({
    "School Name": "count"
})
school_count = school_count.withColumnRenamed( "count(School Name)", "school_count")
school_count.show(1, vertical = True, truncate=100)
school_count.createOrReplaceTempView("school")



# read ERP file and create tempview
ERP_sdf = spark.read.csv(f"{INPUT_DIR}ABS/ERP/ERP.csv", header=True)
ERP_sdf = ERP_sdf.filter(F.col("year") == 2021)
ERP_sdf.show(1, vertical = True, truncate=100)
ERP_sdf.createOrReplaceTempView("ERP")



# read median household income file and create tempview
household_sdf = spark.read.csv(f"{INPUT_DIR}ABS/Household_income/Household_income.csv", header=True)
household_sdf.show(2, vertical = True, truncate=100)
household_sdf.createOrReplaceTempView("household")

# read PTV station file and create tempview
ptv_sdf = spark.read.csv(f"{INPUT_DIR}PTV/public_trans.csv", header=True)
ptv_sdf.show(1, vertical = True, truncate=100)
ptv_sdf.createOrReplaceTempView("ptv")

-RECORD 0-----------------
 SA2          | 209031212 
 school_count | 9         
only showing top 1 row

-RECORD 0---------------
 SA2        | 201011481 
 year       | 2021      
 population | 9656      
only showing top 1 row

-RECORD 0------------------
 SA2           | 213051589 
 median_income | 1862      
-RECORD 1------------------
 SA2           | 209041437 
 median_income | 1979      
only showing top 2 rows

-RECORD 0----------------------------------------------------------------------------------------------------------------
 SA2_CODE21       | 201011001                                                                                            
 geometry         | POLYGON ((143.78282104711133 -37.566657808073295, 143.75557764214773 -37.56346721632544, 143.7480... 
 metrobus_count   | 0                                                                                                    
 metrotrain_count | 0                                                                     

Merge data

In [7]:
# inner join
print(school_count.columns)
print(ERP_sdf.columns)
print(household_sdf.columns)
combine_sdf = spark.sql("""
    SELECT  school.SA2, school.school_count, 
        ERP.population AS ERP_population, median_income, 
        metrobus_count, metrotrain_count, metrotram_count, 
        regbus_count, regcoach_count, regtrain_count, skybus_count
    FROM school
    INNER JOIN ERP ON school.SA2 = ERP.SA2
    INNER JOIN household ON school.SA2 = household.SA2
    INNER JOIN ptv ON school.SA2 = ptv.SA2_CODE21
""")
combine_sdf.show(1, vertical = True, truncate=100)
combine_sdf.write.option("header", True).mode("overwrite").csv(
    f"{OUTPUT_DIR}sa2_info.csv"
)

['SA2', 'school_count']
['SA2', 'year', 'population']
['SA2', 'median_income']
-RECORD 0---------------------
 SA2              | 202011018 
 school_count     | 13        
 ERP_population   | 14951     
 median_income    | 1267      
 metrobus_count   | 0         
 metrotrain_count | 0         
 metrotram_count  | 0         
 regbus_count     | 142       
 regcoach_count   | 2         
 regtrain_count   | 1         
 skybus_count     | 0         
only showing top 1 row



## history_data

#### ERP + interest_rate + price_index -> year, SA2

Read and prepare all data sets and create TempView

In [8]:
# read ERP file and create tempview
ERP_sdf = spark.read.csv(f"{INPUT_DIR}ABS/ERP/ERP.csv", header=True)
ERP_sdf = ERP_sdf.filter(F.col("year") == 2021)
ERP_sdf.show(1, vertical = True, truncate=100)
ERP_sdf.createOrReplaceTempView("ERP")

# read population projection file and create tempview
interest_sdf = spark.read.csv(f"{INPUT_DIR}rba/interest_rate/interest_rate.csv", header=True)
interest_sdf = interest_sdf.filter(F.col("year") == 2021)
interest_sdf.show(1, vertical = True, truncate=100)
interest_sdf.createOrReplaceTempView("interest")

# read property price index file and create tempview
index_sdf = spark.read.csv(f"{INPUT_DIR}ABS/Price_index/Price_index.csv", header=True)
index_sdf = index_sdf.filter(F.col("year") == 2021)
index_sdf.show(1, vertical = True, truncate=100)
index_sdf.createOrReplaceTempView("index")

-RECORD 0---------------
 SA2        | 201011481 
 year       | 2021      
 population | 9656      
only showing top 1 row

-RECORD 0----
 _c0  | 112  
 year | 2021 
 bond | 0.93 

-RECORD 0-----------
 year        | 2021 
 price_index | 185  



Merge data

In [9]:

# inner join
print(ERP_sdf.columns)
print(interest_sdf.columns)
print(index_sdf.columns)
combine_sdf = spark.sql("""
    SELECT  SA2, ERP.year, population, bond, price_index
    FROM ERP
    INNER JOIN interest ON ERP.year = interest.year
    INNER JOIN index ON ERP.year = index.year
""")
combine_sdf.show(1, vertical = True, truncate=100)
combine_sdf.write.option("header", True).mode("overwrite").csv(
    f"{OUTPUT_DIR}history_data.csv"
)

['SA2', 'year', 'population']
['_c0', 'year', 'bond']
['year', 'price_index']
-RECORD 0----------------
 SA2         | 201011481 
 year        | 2021      
 population  | 9656      
 bond        | 0.93      
 price_index | 185       
only showing top 1 row



## Match SA2 to Rental data

In [10]:
rent_df = pd.read_csv(f"../data/raw/rent/rent_raw.csv").reset_index()

In [11]:
# read local files
rent_df = pd.read_csv(f"../data/raw/rent/rent_raw.csv").reset_index()


rent_gdf = gpd.GeoDataFrame(rent_df, geometry=gpd\
        .points_from_xy(rent_df["Longitude"], rent_df["Latitude"]))
boundary_gdf = gpd.read_file(f"../data/raw/ABS/digitalBoundary/SA2_2021_AUST_GDA2020.shp")
rent_gdf["geometry"] = rent_gdf["geometry"].set_crs("epsg:7844")

# feature selction on boundary gdf
boundary_gdf = boundary_gdf.loc[boundary_gdf["STE_NAME21"] == "Victoria"]
boundary_gdf = boundary_gdf[["SA2_CODE21", "geometry"]]
boundary_gdf["SA2_CODE21"] = boundary_gdf["SA2_CODE21"].astype("int64")

# assgin sa2 to rent gdf
join_gdf = gpd.sjoin(rent_gdf, boundary_gdf, how="right")
join_gdf = join_gdf.dropna()
join_gdf["index_left"] = join_gdf["index_left"].astype("int")
rent_gdf = rent_gdf.loc[join_gdf["index_left"]]
join_gdf = join_gdf.reset_index()
rent_gdf["SA2"] = join_gdf[["SA2_CODE21"]]
rent_gdf = rent_gdf.dropna()
rent_gdf["SA2"] = rent_gdf["SA2"].astype("int64")
rent_df = rent_df.loc[rent_gdf["index"]]
rent_df["SA2"] = rent_gdf["SA2"]

rent_df["rent_id"] = range(rent_df.shape[0])
rent_df = rent_df.set_index("rent_id").drop(["index"], axis=1)

rent_df = rent_df.dropna().drop_duplicates()
print(rent_df.head())
print(rent_df.shape)
rent_df.to_csv(f"../data/curated/rent.csv", header=True)


          rent  bedroom  baths  parking  \
rent_id                                   
0        490.0        4      2        2   
1        420.0        4      2        2   
2        520.0        4      2        2   
3        440.0        4      2        2   
4        440.0        4      2        2   

                                                       url   Latitude  \
rent_id                                                                 
0        https://www.domain.com.au/9-kilkenny-drive-alf... -37.563073   
1        https://www.domain.com.au/164-shortridge-drive... -37.547241   
2        https://www.domain.com.au/37-mullingar-drive-a... -37.566319   
3        https://www.domain.com.au/66-lugano-avenue-alf... -37.563453   
4        https://www.domain.com.au/57-dyson-drive-alfre... -37.550549   

          Longitude                     geometry        SA2  
rent_id                                                      
0        143.793875  POINT (143.79387 -37.56307)  206021110  


In [12]:
print(rent_df[["SA2", "Latitude", "Longitude"]].drop_duplicates().shape)

(3748, 3)


In [44]:
def match_sa2_to_data(target_df, output_path):
    # get GeoDataFrame
    target_gdf = gpd.GeoDataFrame(target_df, geometry=gpd.GeoSeries.from_wkt(target_df["geometry"]), crs="epsg:7844")

    # assgin sa2 to rent gdf
    join_gdf = gpd.sjoin(target_gdf, boundary_gdf, how="right")
    join_gdf = join_gdf.dropna()
    join_gdf["index_left"] = join_gdf["index_left"].astype("int")
    target_gdf = target_gdf.loc[join_gdf["index_left"]]
    join_gdf = join_gdf.reset_index()
    target_gdf["SA2"] = join_gdf[["SA2_CODE21"]].astype("int64")
    target_gdf = target_gdf.dropna().drop_duplicates().reset_index(drop=True)
    print(target_gdf.head())
    print(target_gdf.shape)
    target_gdf.to_csv(output_path, header=True)

## Match SA2 to tram_stop data

In [46]:
tram_stop_df = pd.read_csv(f"../data/raw/tram_stop.csv", header=None)
tram_stop_df = tram_stop_df[[0, 3]]
tram_stop_df.columns = ["geometry", "tram_stop"]
tram_stop_outpath = "../data/curated/tram_stop_sa2.csv"

match_sa2_to_data(tram_stop_df, tram_stop_outpath)

                      geometry                                 tram_stop  \
0  POINT (144.95357 -37.82154)  Victoria Police Centre / Flinders Street   
1  POINT (144.95110 -37.82190)           The Goods Shed / Wurundjeri Way   
2  POINT (144.94773 -37.82216)        Docklands Park / Harbour Esplanade   
3  POINT (144.94651 -37.81866)         Bourke Street / Harbour Esplanade   
4  POINT (144.94512 -37.81543)          Central Pier / Harbour Esplanade   

         SA2  
0  206041118  
1  206041118  
2  206041118  
3  206041119  
4  206041119  
(28, 3)


## Match SA2 to station data

In [48]:
station_df = pd.read_csv(f"../data/raw/station.csv", header=None)
station_df = station_df[[0, 1]]
station_df.columns = ["station", "geometry"]
station_outpath = "../data/curated/station_sa2.csv"

match_sa2_to_data(station_df, station_outpath)


     station                     geometry        SA2
0     Anstey  POINT (144.96056 -37.76190)  206011497
1  Brunswick  POINT (144.95956 -37.76794)  206041127
2     Jewell  POINT (144.95875 -37.77485)  208041194
3     Coburg  POINT (144.96335 -37.74241)  206061138
4   Moreland  POINT (144.96185 -37.75434)  210031236
(219, 3)


## Match SA2 to parking_spot data

In [50]:
parking_spot_df = pd.read_csv(f"../data/raw/parking_spot.csv", header=None)
parking_spot_df = parking_spot_df[[0, 6]]
parking_spot_df.columns = ["geometry", "parking_spot"]
parking_spot_outpath = "../data/curated/parking_spot_sa2.csv"

match_sa2_to_data(parking_spot_df, parking_spot_outpath)


                                            geometry  \
0  MULTIPOLYGON (((144.97048 -37.79701, 144.97046...   
1  MULTIPOLYGON (((144.96041 -37.80303, 144.96040...   
2  MULTIPOLYGON (((144.97418 -37.79577, 144.97419...   
3  MULTIPOLYGON (((144.96450 -37.80544, 144.96449...   
4  MULTIPOLYGON (((144.96220 -37.80276, 144.96219...   

                                        parking_spot          SA2  
0  Rathdowne Street between Elgin Street and Palm...  206041117.0  
1  Leicester Street between Queensberry Street an...  206041117.0  
2  Station Street between Palmerston Street and K...  206041117.0  
3  Cardigan Street between Victoria Street and Qu...  206041117.0  
4  Bouverie Street between Lincoln Square South a...  206041117.0  
(22573, 3)
